In [ ]:
import numpy as np  
from poliastro.bodies import Moon 
from poliastro.twobody.orbit import Orbit
from poliastro.bodies import Moon 
from astropy import units as u
from poliastro.core.angles import nu_to_E, E_to_M
import sympy as sym

global MU

MU = Moon.k.to(u.km**3/u.s**2)
MU = MU.to_value()


def phi_motion(xx,yy,zz, as_numpy=True):
     x, y, z = sym.symbols('x,y,z')              # componenti posizione (vettore r)
     mi = sym.symbols(r'\mu')             
     r = sym.sqrt(x**2 + y**2 + z**2)

     f00 = ((3*mi*(x**2))/(r**5)) - mi/(r**3)
     f01 = (3*mi*x*y)/(r**5)
     f02 = (3*mi*x*z)/(r**5) 

     f10 = (3*mi*y*x)/(r**5)
     f11 = ((3*mi*(y**2))/(r**5)) - mi/(r**3) 
     f12 = (3*mi*y*z)/(r**5) 

     f20 = (3*mi*x*z)/(r**5)
     f21 = (3*mi*z*y)/(r**5) 
     f22 = ((3*mi*(z**2))/(r**5)) - mi/(r**3) 
     motion = sym.Matrix(([f00, f01, f02], [f10, f11, f12], [f20, f21, f22]))

     phi = sym.Matrix([[sym.zeros(3,3),sym.eye(3,3)], [motion, sym.zeros(3,3)]])
     phi = phi.subs(x,xx).subs(y,yy).subs(z,zz).subs(mi,MU)
     phi = phi.evalf(maxn=300)
     if as_numpy:
          phi = np.array(phi).astype(np.float64)
     return phi


In [ ]:
phi=phi_motion(180,-1780, 0)

In [ ]:
import sympy as sym

In [ ]:
sym.symbols('omega')

In [12]:
from Filters.kalman import KalmanFilter
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
from Filters.STM import Keplerian

# DB = pd.read_csv("orbite/lunar_crater_database_robbins_2018.csv")
# Init:
df_pos = pd.read_csv('orbite/Position_Fixed.csv')
time = df_pos['Time (UTCG)']
real_X, real_Y, real_Z = df_pos['x (km)'].to_numpy(), df_pos['y (km)'].to_numpy(), df_pos['z (km)'].to_numpy()
#Quaternione e velocita' angolare 
dq = pd.read_csv("orbite/Attitude_Quaternions_Fixed.csv")  
real_q1, real_q2, real_q3, real_q4 = dq['q1'].to_numpy(), dq['q2'].to_numpy(), dq['q3'].to_numpy(), dq['q4'].to_numpy()
real_om1, real_om2, real_om3 = dq['wx (deg/sec)'].to_numpy(), dq['wy (deg/sec)'].to_numpy(), dq['wz (deg/sec)'].to_numpy()
#Velocita'
dvf = pd.read_csv("orbite/Velocity_Fixed.csv") 
real_Vxs, real_Vys, real_Vzs = dvf['vx (km/sec)'].to_numpy(), dvf['vy (km/sec)'].to_numpy(),dvf['vz (km/sec)'].to_numpy()
skip=0
position = [real_X[0+skip],real_Y[0+skip],real_Z[0+skip]]
velocity = [real_Vxs[0+skip], real_Vys[0+skip], real_Vzs[0+skip]]
attitude = [real_q1[0+skip], real_q2[0+skip], real_q3[0+skip], real_q4[0+skip]]
rotation = [real_om1[0+skip], real_om2[0+skip], real_om3[0+skip]]
velocity = [-x for x in velocity]

In [2]:
F = Keplerian(position, velocity, attitude, rotation, dt=10, as_numpy=False)
F

array([[1.00000000000000, 0, 0, 15.0000000000000, 0, 0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0],
       [0, 1.00000000000000, 0, 0, 15.0000000000000, 0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0],
       [0, 0, 1.00000000000000, 0, 0, 15.0000000000000, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0],
       [-8.48399456663518e-6, -1.61580681440992e-6, 0, 1.00000000000000,
        0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [-1.61580681440992e-6, 1.70697758083938e-5, 0, 0,
        1.00000000000000, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0, 0, -8.58575938824356e-6, 0, 0, 1.00000000000000, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.00000000000000,
        -2.49999687500000e-5, 0.268418865405000, 1.33147231753125,
        -2.75955836382000, -2.70209864073875],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.313100745203619,
        0.342234994763699, 0.723411972869111, 3.85256163618000,
        1.33147231753125, -1.35197232634875],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.261

In [13]:
import sympy as sym  
q1,q2,q3,q4 = sym.symbols('q_1,q_2,q_3,q_4')
unitariety = sym.Eq(sym.sqrt(q1*q1+q2*q2+q3*q3+q4*q4),1)
solutions = sym.solve(unitariety, q4)
s1,s2 = solutions

In [19]:
solutions

[-sqrt(-q_1**2 - q_2**2 - q_3**2 + 1), sqrt(-q_1**2 - q_2**2 - q_3**2 + 1)]

In [15]:
s1.subs(q1,real_q1[0]).subs(q2,real_q2[0]).subs(q3,real_q3[0])

-0.250598213732660

In [16]:
s2.subs(q1,real_q1[0]).subs(q2,real_q2[0]).subs(q3,real_q3[0])

0.250598213732660

In [17]:
real_q4[0]

0.250595